# Get Kroger grocery store locations

#### Load Python tools and Jupyter config

In [1]:
import us
import json
import requests
import pandas as pd
import jupyter_black
import altair as alt
import geopandas as gpd
from bs4 import BeautifulSoup
from vega_datasets import data
from tqdm.notebook import tqdm, trange
from json.decoder import JSONDecodeError

In [2]:
jupyter_black.load()
pd.options.display.max_columns = 100
pd.options.display.max_rows = 100

DataTransformerRegistry.enable('default')

In [3]:
place = "kroger"
place_formal = "Kroger"
color = "#084999"

## Scrape

#### Let's limit our questions. First, in how many states does Kroger operate?

In [4]:
headers = {
    "authority": "www.kroger.com",
    "accept": "application/json, text/plain, */*",
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36",
}

#### Get list of states from [Kroger directory](https://www.kroger.com/stores/grocery)

In [5]:
dir_response = requests.get("https://www.kroger.com/stores/grocery", headers=headers)
dir_soup = BeautifulSoup(dir_response.text, "html.parser")

#### Get all the links on the page and convert urls to unique list of state abbreviations

In [6]:
hrefs = [a["href"] for a in dir_soup.find_all("a", class_="kds-Link kds-Link--inherit")]

# Extract state abbreviations from hrefs
state_abbr_set = set()
states = []

for href in hrefs:
    parts = href.split("/")
    if len(parts) >= 3:
        state_abbr = parts[3].upper()
        if state_abbr not in state_abbr_set:
            states.append(state_abbr)
            state_abbr_set.add(state_abbr)

#### Now, get only the ZIP Codes from Kroger states in order to craft our request

In [7]:
zips = (
    pd.read_csv("../../_reference/data/zips_reference.csv")
    .query(f"pop2010 > 25000 and state.isin({states})")
    .sort_values("pop2010", ascending=False)
    .reset_index(drop=True)[["zip", "pop2010", "po_name", "state"]]
)

In [ ]:
len(zips['state'].unique())

#### That leaves a limited number of ZIP Codes. We can now query Kroger's API with them all. 

In [9]:
zips_list = list(zips["zip"].unique())
len(zips_list)

#### Send a request for each zip code, filter out other company's under Kroger's parent, and store in a dataframe

In [11]:
response_list = []

for z in tqdm(zips_list):

    headers = {
        "authority": "www.kroger.com",
        "accept": "application/json, text/plain, */*",
        "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36",
    }

    params = {
        "filter.query": f"{z}",
        "projections": "full",
    }

    response = requests.get(
        "https://www.kroger.com/atlas/v1/stores/v2/locator",
        params=params,
        headers=headers,
    )

    try:
        stores = response.json()["data"]["stores"]
    except JSONDecodeError as e:
        continue

    if not stores:
        continue

    # Filter out non-Kroger stores
    kroger_stores = [store for store in stores if store.get("brand") == "KROGER"]

    response_list.extend(kroger_stores)

# Convert to DataFrame
src = pd.DataFrame(response_list)

  0%|          | 0/1619 [00:00<?, ?it/s]

#### Deal with nested columns

In [ ]:
src["phone"] = pd.json_normalize(src["phoneNumber"])["pretty"]

In [ ]:
src[
    [
        "address.addressLines",
        "address.cityTown",
        "address.name",
        "address.postalCode",
        "address.stateProvince",
        "address.residential",
        "address.countryCode",
        "location.lat",
        "location.lng",
    ]
] = pd.json_normalize(src["locale"])

#### Just get the columns we need

In [191]:
df = (
    src[
        [
            "brand",
            "locale",
            "phoneNumber",
            "vanityName",
            "storeType",
            "locationId",
            "legalName",
            "facilityName",
            "tz",
            "address.addressLines",
            "address.cityTown",
            "address.postalCode",
            "address.stateProvince",
            "address.residential",
            "address.countryCode",
            "address.name",
            "location.lat",
            "location.lng",
        ]
    ]
    .drop(
        [
            "locale",
            "phoneNumber",
            "address.residential",
            "address.countryCode",
            "facilityName",
            "address.name",
        ],
        axis=1,
    )
    .rename(
        columns={
            "storeType": "type",
            "vanityName": "store_name",
            "address.addressLines": "address",
            "address.cityTown": "city",
            "address.postalCode": "zip",
            "address.stateProvince": "state",
            "location.lat": "latitude",
            "location.lng": "longitude",
            "locationId": "store_number",
            "tz": "timezone",
        }
    )
    .drop_duplicates(subset="store_number")
    .copy()
)

In [192]:
df["address"] = df["address"].apply(pd.Series)

---

#### Create a mapping of state abbreviations to full state names using the us library

In [193]:
state_mapping = {state.abbr: state.name for state in us.states.STATES}

#### New column of full state names based on abbreviations

In [194]:
df["state_name"] = df["state"].map(state_mapping)
df["brand"] = df["brand"].str.title()

#### Define types (work in progress)

In [195]:
types = {
    "M": "Marketplace",
    "Q": "Fresh Fare",
    "PH": "Pharmacy",
    "I": "I",
    "C": "C",
    "S": "S",
    "F": "F",
}

In [196]:
df["type_desc"] = df["type"].map(types)

In [197]:
df.type.value_counts()

type
C     1025
M      131
S       54
I       12
F       11
Q        7
PH       4
PI       2
Name: count, dtype: int64

In [200]:
df.query('type=="C"').head()

,brand,store_name,type,store_number,legalName,timezone,address,city,zip,state,latitude,longitude,state_name,type_desc
1,Kroger,Barker Cypress,C,03400352,Kroger Texas LP,America/Chicago,18030 FM 529 Rd,Cypress,77433,TX,29.881445,-95.685560,Texas,C
2,Kroger,Eagle Ranch,C,03400378,Kroger Texas LP,America/Chicago,6055 Fry Rd,Katy,77449,TX,29.860067,-95.721681,Texas,C
3,Kroger,Eldridge Lakes Town Center,C,03400396,Kroger Texas LP,America/Chicago,6350 N Eldridge Pkwy,Houston,77041,TX,29.865399,-95.606798,Texas,C
4,Kroger,Easton Commons,C,03400374,Kroger Texas LP,America/Chicago,8550 Highway 6 N,Houston,77095,TX,29.902531,-95.631714,Texas,C
7,Kroger,Briar Forest,C,03400349,Kroger Texas LP,America/Chicago,1520 Eldridge Pkwy,Houston,77077,TX,29.757519,-95.627398,Texas,C


In [180]:
len(df)

1244

---

## Geography

#### Make it a geodataframe

In [69]:
df_geo = df.copy()

In [70]:
gdf = gpd.GeoDataFrame(
    df_geo, geometry=gpd.points_from_xy(df_geo.longitude, df_geo.latitude)
)

In [71]:
locations_gdf = gdf.set_crs("EPSG:4326").copy()

---

## Maps

#### US states background

In [72]:
background = (
    alt.Chart(alt.topo_feature(data.us_10m.url, feature="states"))
    .mark_geoshape(fill="#e9e9e9", stroke="white")
    .properties(width=800, height=500, title=f"{place_formal} locations")
    .project("albersUsa")
)

#### Location points map

In [78]:
points = (
    alt.Chart(gdf)
    .mark_circle(size=5, color=color)
    .encode(
        longitude="longitude:Q",
        latitude="latitude:Q",
    )
)

point_map = background + points
point_map.configure_view(stroke=None)

alt.LayerChart(...)

#### Location proportional symbols map

In [74]:
symbols = (
    alt.Chart(gdf)
    .transform_aggregate(
        latitude="mean(latitude)",
        longitude="mean(longitude)",
        count="count()",
        groupby=["state"],
    )
    .mark_circle()
    .encode(
        longitude="longitude:Q",
        latitude="latitude:Q",
        size=alt.Size("count:Q", title="Count by state"),
        color=alt.value(color),
        tooltip=["state:N", "count:Q"],
    )
    .properties(
        title=f"Number of {place_formal} in US, by average lon/lat of locations"
    )
)

symbol_map = background + symbols
symbol_map.configure_view(stroke=None)

alt.LayerChart(...)

---

## Exports

#### JSON

In [75]:
df.to_json(
    f"data/processed/{place.lower().replace(' ', '_')}_locations.json",
    indent=4,
    orient="records",
)

#### CSV

In [76]:
df.to_csv(
    f"data/processed/{place.lower().replace(' ', '_')}_locations.csv", index=False
)

#### GeoJSON

In [77]:
locations_gdf.to_file(
    f"data/processed/{place.lower().replace(' ', '_')}_locations.geojson",
    driver="GeoJSON",
)